In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.7 MB/s eta 0:00:00


In [36]:
from transformers import AutoTokenizer, BertModel, GPT2LMHeadModel, GPT2Tokenizer
from transformers import TrainingArguments, Trainer
from transformers import GPT2Config, GPT2Model
from torch.utils.data import random_split

# from datasets import load_dataset
import torch.optim as optim
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# assert device == torch.device('cuda'), "Not using CUDA. Set: Runtime > Change runtime type > Hardware Accelerator: GPU"
import math
import time
import sys
import json
import numpy as np


def main():  

    torch.manual_seed(0)
    answers = ['A','B','C','D']

    train = []
    test = []
    valid = []
    
    file_name = 'train_complete.jsonl'        
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)
        
        base = result['fact1'] + ' [SEP] ' + result['question']['stem']
        ans = answers.index(result['answerKey'])
        
        obs = []
        for j in range(4):
            text = base + result['question']['choices'][j]['text'] + ' [SEP]'
            if j == ans:
                label = 1
            else:
                label = 0
            obs.append([text, label])
        train.append(obs)
        
        # print(obs)
        # print(' ')
        
        # print(result['question']['stem'])
        # print(' ',result['question']['choices'][0]['label'],result['question']['choices'][0]['text'])
        # print(' ',result['question']['choices'][1]['label'],result['question']['choices'][1]['text'])
        # print(' ',result['question']['choices'][2]['label'],result['question']['choices'][2]['text'])
        # print(' ',result['question']['choices'][3]['label'],result['question']['choices'][3]['text'])
        # print('  Fact: ',result['fact1'])
        # print('  Answer: ',result['answerKey'])
        # print('  ')
                
    file_name = 'dev_complete.jsonl'        
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)
        
        base = result['fact1'] + ' [SEP] ' + result['question']['stem']
        ans = answers.index(result['answerKey'])
        
        obs = []
        for j in range(4):
            text = base + result['question']['choices'][j]['text'] + ' [SEP]'
            if j == ans:
                label = 1
            else:
                label = 0
            obs.append([text,label])
        valid.append(obs)
        
    file_name = 'test_complete.jsonl'        
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)
        
        base = result['fact1'] + ' [SEP] ' + result['question']['stem']
        ans = answers.index(result['answerKey'])
        
        obs = []
        for j in range(4):
            text = base + result['question']['choices'][j]['text'] + ' [SEP]'
            if j == ans:
                label = 1
            else:
                label = 0
            obs.append([text,label])
        test.append(obs)

    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")
    optimizer = optim.Adam(model.parameters(), lr=3e-5)
    linear = torch.rand(768,2)
    
#    Add code to fine-tune and test your MCQA classifier.
           
                 
if __name__ == "__main__":
    main()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
def trainSet():
    answers = ['A','B','C','D']

    train = []
    test = []
    valid = []
    
    file_name = 'train_complete.jsonl'        
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)
        
        base = result['fact1'] + ' ' + result['question']['stem'] + ' '
        ans = answers.index(result['answerKey'])
        
        obs = []
        for j in range(4):
            text = base + ' ' + result['question']['choices'][j]['label'] + ' ' + result['question']['choices'][j]['text'] + ' <answer>'
            if j == ans:
                label = 1
            else:
                label = 0
            obs.append([text, result['answerKey']])
        train.append(obs)
        
        # print(obs)
        # print(' ')
        
        # print(result['question']['stem'])
        # print(' ',result['question']['choices'][0]['label'],result['question']['choices'][0]['text'])
        # print(' ',result['question']['choices'][1]['label'],result['question']['choices'][1]['text'])
        # print(' ',result['question']['choices'][2]['label'],result['question']['choices'][2]['text'])
        # print(' ',result['question']['choices'][3]['label'],result['question']['choices'][3]['text'])
        # print('  Fact: ',result['fact1'])
        # print('  Answer: ',result['answerKey'])
        # print('  ')
                
    file_name = 'dev_complete.jsonl'        
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)
        
        base = result['fact1'] + ' [SEP] ' + result['question']['stem'] + ' '
        ans = answers.index(result['answerKey'])
        
        obs = []
        for j in range(4):
            text = base + ' ' + result['question']['choices'][j]['label'] + ' ' + result['question']['choices'][j]['text'] + ' <answer>'
            if j == ans:
                label = 1
            else:
                label = 0
            obs.append([text, result['question']['choices'][j]['label']])
        valid.append(obs)
        
    file_name = 'test_complete.jsonl'        
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)
        
        base = result['fact1'] + ' [SEP] ' + result['question']['stem'] + ' '
        ans = answers.index(result['answerKey'])
        
        obs = []
        for j in range(4):
            text = base + ' ' + result['question']['choices'][j]['label'] + ' ' + result['question']['choices'][j]['text'] + ' <answer>'
            if j == ans:
                label = 1
            else:
                label = 0
            obs.append([text, result['question']['choices'][j]['label']])
        test.append(obs)

    return train, valid, test


In [38]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer("<answer>")['input_ids']


train, valid, test = trainSet()

In [35]:
def encodeTxt(tokenizer, val):
  return tokenizer.encode(val, return_tensors = 'pt')

out = './output'
lr = 0.001
training_args = TrainingArguments(
    output_dir=out,          
    num_train_epochs=6,          
    save_steps = 1000,
    learning_rate = lr,
    weight_decay=0.005,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4
)

In [73]:
class inputGen():
    def __init__(self, x, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        tokenizer.pad_token = tokenizer.eos_token
        for txt in x:
            encodings_dict = tokenizer(txt[0][0], padding='max_length', max_length=max_length)
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [74]:
# # trainer = Trainer(
# #     train_dataset = train_ds,         
# #     eval_dataset = val_ds,                        
# #     args=training_args,         
# #     model=base_model           
# # )

# # trainer.train()
max_length = max([len(tokenizer.encode(x[0][0])) for x in train])
# print(max_length)
from torch.utils.data import random_split

X_train = inputGen(train, tokenizer, max_length)

# print(X_train)

# for x in train:
#     print(tokenizer.encode(x[0]))
# max_length

train_size = int(0.8 * len(X_train))
train_dataset, val_dataset = random_split(X_train, [train_size, len(X_train) - train_size])

In [75]:
train_ds = []
val_ds = []

for idx, val in enumerate(train_dataset):
  train_val = {}
  train_val['input_ids'] = val[0]
  train_val['attention_mask'] = val[1]
  train_val['labels'] = tokenizer.encode(train[idx][1])
  train_ds.append(train_val)

for idx, val in enumerate(val_dataset):
  train_val = {}
  train_val['input_ids'] = val[0]
  train_val['attention_mask'] = val[1]
  train_val['labels'] = tokenizer.encode(train[idx][1])
  val_ds.append(train_val)


In [79]:
out = './output'
lr = 0.001
base_model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir=out,          
    num_train_epochs=6,          
    save_steps = 1000,
    learning_rate = lr,
    weight_decay=0.005,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4
)

trainer = Trainer(
    train_dataset = train_ds,         
    eval_dataset = val_ds,                        
    args=training_args,         
    model=base_model           
)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.26.1",
  "use_cach

OutOfMemoryError: ignored